## League of Legends - Data Processing Notebook

Purpose: This notebook documents a clear and reproducible Data Processing workflow for the League of Legends Match Prediction challange. It explains why each step is needed, the code to collect and transform Riot match data, and produces a clean dataset ready for modeling.

## Introduction & Goals

**Goals of this notebook:**

-   Collect match data from the Riot API.

-   Parse and extract both pre-game features (champion picks, roles) and early-game features at 10 minutes (gold diff, kills, objectives).

-   Produce a cleaned, versioned dataset (CSV) with clear documentation for each column.

**Why separate processing from modeling?**

Riot match timelines and match lists can become large (MBs–GBs). Processing and cleaning in a dedicated notebook or script avoids repeated heavy work during model experimentation and keeps the ML notebook focused on training and evaluation.

## Environment & setup

In [1]:
# Import core Python libraries
import pandas as pd
import requests
import tqdm
import pyarrow
import fastparquet
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import json
from typing import Dict, Any, List
from dotenv import load_dotenv

import requests
import pandas as pd
from tqdm import tqdm


# plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(context="notebook", style="darkgrid", palette="deep", font="sans-serif", font_scale=1)


# Load environment variables
from dotenv import load_dotenv
load_dotenv()


RIOT_API_KEY = os.getenv('RIOT_API_KEY')
if not RIOT_API_KEY:
    raise RuntimeError('Set RIOT_API_KEY as an environment variable or in a .env file')


HEADERS = {"X-Riot-Token": RIOT_API_KEY}

## Data sourcing (Riot API overview)

### What endpoints we be used:

- **Match IDs** by **PUUID:** /lol/match/v5/matches/by-puuid/{puuid}/ids - returns a list of match IDs for a given player.

- **Match detail by matchId**: /lol/match/v5/matches/{matchId} — returns the full match info and timeline (timelines might be included alongside).

### Filtering notes:

- Use **queue=420** to restrict to **Ranked Solo/Duo matches (competitive).**

- Filter out **remakes** or **extremely short games** (e.g., gameDuration < 9 minutes) to avoid noisy examples.

### Rate limits & best practices:

- Riot enforces **rate limits**, so I will be using **time.sleep** to pause between requests and wait longer if blocked (HTTP 429).

- **Cache** raw match JSONs to disk so you don't re-download them.

## Data requirements and Schema


### 1. Core Features (and why)

- **Match metadata**: `matchId`, `queueId`, `gameDuration` - needed to filter matches.
- **Champion & role picks**: champion IDs/names, team positions - needed for pre-game features.
- **Lane metrics at 10 minutes**: gold diff, CS diff, XP diff, K/D/A per lane - strong early indicators.
- **Team objectives at 10 minutes**: `firstBlood`, `firstTower`, `firstDragon` (+ type), `firstHerald`, counts of towers/dragons.
- **Target variables**: `blue_win` (0/1) and `win_probability` (for model output).



### 2. Proposed Output Schema (one row per match)

### Match Metadata
- `matchId` (`str`)
- `queueId` (`int`)
- `gameDuration` (`int`, seconds)
- `blue_win` (`0/1`)

### Champion Picks & Roles
- `blue_champions` (list of champion IDs)
- `red_champions` (list of champion IDs)
- `blue_roles` (list of positions)
- `red_roles` (list of positions)

### Early Features (10 min)
- `blue_gold_10`, `red_gold_10`, `gold_diff_10` - difference in gold at 10 minutes
- `blue_cs_10`, `red_cs_10`, `cs_diff_10` - difference in farm at 10 minutes
- `blue_xp_10`, `red_xp_10`, `xp_diff_10` - difference in xp at 10 minutes
- `blue_kills_10`, `red_kills_10`, `kills_diff_10`- difference in kills at 10 minutes

### Objectives (10 min)
- `first_blood` (None / blue / red)
- `first_tower` (None / blue / red)
- `first_dragon` (None / blue / red)
- `first_rift_herald` (None / blue / red)
- `blue_towers_10`, `red_towers_10` - tower kills at 10 minutes
- `blue_dragons_10`, `red_dragons_10` - dragon kills at 10 minutes


## Storage strategy & versioning

For this project, I decided to keep storage dead simple.
### Raw and Processed Data
- **Raw data**: every match JSON from the Riot API lives in `data/raw/` (both the match detail and its timeline). I never overwrite these so I can always reprocess.
- **Processed data**: two Parquet files:
  - `data/processed/lol_pregame_data.parquet` (lane champion picks)
  - `data/processed/lol_10min_data.parquet` (early game stats + first objectives)
- **Preview CSVs**: `lol_pregame_data_preview.csv` and `lol_10min_data_preview.csv` (just first 50 rows to eyeball).

Why Parquet? Smaller + keeps types + faster to load. CSV is only for quick inspection.

No manifest / no versioning right now — I just overwrite while iterating. If I later need reproducibility or experiment tracking, I can start saving versioned snapshots like `lol_pregame_data_v002.parquet`.

## Raw data collection

### Important initial design choices:

- I am going to collect match IDs by querying several PUUIDs (players). Prefer high-activity public accounts.

- For initial proof-of-concept, I will collect 1,000–5,000 matches. For final model I will have tens of thousands

In [55]:
# Constants + basic Riot fetch helpers (simple version)
REGION = 'europe'  # riot regional routing
HEADERS = {"X-Riot-Token": RIOT_API_KEY}  # auth header
MATCHLIST_URL = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{{puuid}}/ids'
MATCH_URL = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/{{matchId}}'
TIMELINE_URL = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/{{matchId}}/timeline'
RAW_DIR = 'data/raw'

# Basic GET with tiny retry + rate limit handling
def safe_get(url, params=None, retries=5):
    for i in range(retries):
        resp = requests.get(url, headers=HEADERS, params=params)
        if resp.status_code == 200:
            return resp.json()
        elif resp.status_code == 429:  # rate limited
            wait = int(resp.headers.get('Retry-After', 1))
            print(f"Rate limited. Waiting {wait}s...")
            time.sleep(wait)
    raise RuntimeError(f"Failed to GET {url} after {retries} retries")

# Get player unique id from riot name + tag
def get_puuid(game_name, tag_line):
    url = f"https://{REGION}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    resp = requests.get(url, headers=HEADERS)
    if resp.status_code == 200:
        return resp.json().get("puuid")
    print(f"Failed to get PUUID for {game_name}#{tag_line}: {resp.status_code}")
    return None

# Just grab some match ids for one player (solo queue)
def fetch_match_ids(puuid, count=20, queue=420):
    params = {"start": 0, "count": count, "queue": queue}
    return safe_get(MATCHLIST_URL.format(puuid=puuid), params=params)

# Save both match + timeline JSON locally if missing (cache)
def fetch_and_save_match_with_timeline(match_id):
    os.makedirs(RAW_DIR, exist_ok=True)

    match_path = os.path.join(RAW_DIR, f"{match_id}.json")
    if not os.path.exists(match_path):  # don't redownload
        match_data = safe_get(MATCH_URL.format(matchId=match_id))
        with open(match_path, 'w', encoding='utf-8') as f:
            json.dump(match_data, f)

    timeline_path = os.path.join(RAW_DIR, f"{match_id}_timeline.json")
    if not os.path.exists(timeline_path):
        timeline_data = safe_get(TIMELINE_URL.format(matchId=match_id))
        with open(timeline_path, 'w', encoding='utf-8') as f:
            json.dump(timeline_data, f)

    return match_path, timeline_path

# Generic fetch/save helper
def fetch_and_save_json(url, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    if os.path.exists(path):  # already there
        return path
    data = safe_get(url)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f)
    return path

# Single test player (can add more later)
players = [{"name": "BB99", "tag": "BLZNT"}]

# Resolve their PUUIDs
for p in players:
    p["puuid"] = get_puuid(p["name"], p["tag"])
puuids = [p["puuid"] for p in players if p.get("puuid")]
print("PUUIDs fetched:", puuids)

# Rate limiting guard (rough)
all_match_ids = []
MAX_REQUESTS = 99  # under 100 per 2 min window
WINDOW = 120
request_count = 0
start_time = time.time()

for puuid in puuids:
    match_ids = fetch_match_ids(puuid, count=20)  # small sample
    all_match_ids.extend(match_ids)
    
    for match_id in tqdm(match_ids, desc=f"Downloading matches for {puuid}"):
        # Full match
        fetch_and_save_json(MATCH_URL.format(matchId=match_id), os.path.join(RAW_DIR, f"{match_id}.json"))
        # Timeline
        fetch_and_save_json(TIMELINE_URL.format(matchId=match_id), os.path.join(RAW_DIR, f"{match_id}_timeline.json"))
        
        request_count += 2  # two calls per match
        if request_count >= MAX_REQUESTS:
            elapsed = time.time() - start_time
            sleep_time = max(0, WINDOW - elapsed)
            if sleep_time > 0:
                print(f"Sleeping {sleep_time:.1f}s to avoid rate limit...")
                time.sleep(sleep_time)
            start_time = time.time()
            request_count = 0

# Persist the collected list
with open("matchlist.json", "w") as f:
    json.dump(all_match_ids, f, indent=2)

print(f" Downloaded {len(all_match_ids)} matches to '{RAW_DIR}/'")

PUUIDs fetched: ['T9lGIR8iroZCD7e9-3hWNs-wg9h3eA1UjcT_4YsKlkdZY0L9tZWhJlMg9kGT99wpuBNZxT5iDpY3lg']


 Downloaded 20 matches to 'data/raw/'


### Regenerate matchlist.json after adding new matches

In [56]:
# Regenerate matchlist.json from all match JSON files in data/raw
import os
import json

raw_dir = 'data/raw'
match_ids = []
for fname in os.listdir(raw_dir):
    if fname.endswith('.json') and not fname.endswith('_timeline.json'):
        match_id = fname.replace('.json', '')
        match_ids.append(match_id)

# Remove duplicates, just in case
match_ids = sorted(set(match_ids))

with open("matchlist.json", "w") as f:
    json.dump(match_ids, f, indent=2)

print(f"Updated matchlist.json with {len(match_ids)} match IDs from {raw_dir}")

Updated matchlist.json with 45 match IDs from data/raw


## Processing & feature extraction

### Design approach (how I actually set this up)

To keep it simple the data is split into two parts based on when the state of the game:

1. **Pregame table** – only info known before the game starts:
   - Champion picks by lane (top, jg, mid, bot, support) for each team.
   - Match duration (just for quick filtering later).

2. **10-minute table** – snapshot of early game:
   - First objective takers (`tower`, `dragon`, `herald`, grubs = `horde`).
   - Team gold / xp / cs / kills and their diffs at ~10:00.
   - Games shorter than 10 minutes are skipped (remakes / noise).

Both tables have one row per `matchId`, so joining them later is trivial.

Why bother splitting? Faster iteration. I can train a draft-only model without loading timeline data, and I can tweak early game logic without touching the pregame export.

Storage:
- Parquet main files: `lol_pregame_data.parquet` and `lol_10min_data.parquet`.
- Small CSV previews (`*_preview.csv`) just for a quick look.

No manifest, no version bumps — if I change logic I just overwrite the files (this is small scale). If I later need history I can start versioning again.

Flow I run:
1. Regenerate `matchlist.json` from raw if needed.
2. Build pregame (no timeline needed).
3. Build 10m table (needs timeline + duration filter).
4. Save Parquet + preview CSVs.

In [58]:
#Setup for loading previously downloaded JSON files

RAW_DIR = 'data/raw'

# Load a stored match detail JSON by match_id.
def load_match_json(match_id, raw_dir: str = RAW_DIR):
    path = os.path.join(raw_dir, f"{match_id}.json")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Match file not found: {path}")
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Load a stored timeline JSON by match_id.
def load_timeline_json(match_id, raw_dir: str = RAW_DIR):
    path = os.path.join(raw_dir, f"{match_id}_timeline.json")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Timeline file not found: {path}")
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# If aggregate_participant_stats_at_10min not defined yet, create a minimal placeholder to avoid NameError.
# Replace later with real logic if already implemented elsewhere.
try:
    aggregate_participant_stats_at_10min
except NameError:
    def aggregate_participant_stats_at_10min(match_json, timeline_json):
        """Placeholder aggregator: returns basic dict with zeroed stats for each participantId.
        Replace with real implementation that inspects frames around 10:00.
        """
        participants = match_json['info']['participants']
        out = {}
        for p in participants:
            pid = p['participantId']
            out[pid] = {
                'totalGold': p.get('goldEarned', 0),  # fallback to final gold if no timeline logic
                'xp': p.get('champExperience', 0), # fallback to final xp if no timeline logic
                'minionsKilled': p.get('totalMinionsKilled', 0) + p.get('neutralMinionsKilled', 0), # fallback to final if no timeline logic
                'kills': p.get('kills', 0) # fallback to final kills if no timeline logic
            }
        return out
    print("[WARN] Using placeholder aggregate_participant_stats_at_10min. Implement proper 10m snapshot logic later.")

## Pregame Table

In [61]:
# Build Pregame Table

processed_dir = 'data/processed'
os.makedirs(processed_dir, exist_ok=True)  # make sure folder exists

# Load list of match IDs downloaded
with open('matchlist.json','r') as f:
    match_ids_all = json.load(f)

# Map Riot role labels to shorter ones
lane_map = {
    'TOP': 'Top',
    'JUNGLE': 'Jg',
    'MIDDLE': 'Mid',
    'BOTTOM': 'Bot',
    'UTILITY': 'Support'
}

pregame_rows = []

for mid in tqdm(match_ids_all, desc='Pregame (all lanes)'):
    m = load_match_json(mid)  # raw match json
    info = m['info']
    participants = info['participants']
    # placeholders for each lane (filled in later)
    bluetop = bluejg = bluemid = bluebot = bluesupport = ''
    redtop = redjg = redmid = redbot = redsupport = ''
    for p in participants:
        pos = p.get('teamPosition') # TOP, JUNGLE, MIDDLE, BOTTOM, SUPPORT
        team_id = p.get('teamId')
        if pos in lane_map:
            lane_short = lane_map[pos]
            champ = p.get('championName')
            # assign champion to first empty slot for that lane
            if team_id == 100:  # blue side
                if lane_short == 'top' and not bluetop: bluetop = champ
                elif lane_short == 'jg' and not bluejg: bluejg = champ
                elif lane_short == 'mid' and not bluemid: bluemid = champ
                elif lane_short == 'bot' and not bluebot: bluebot = champ
                elif lane_short == 'support' and not bluesupport: bluesupport = champ
            elif team_id == 200:  # red side
                if lane_short == 'top' and not redtop: redtop = champ
                elif lane_short == 'jg' and not redjg: redjg = champ
                elif lane_short == 'mid' and not redmid: redmid = champ
                elif lane_short == 'bot' and not redbot: redbot = champ
                elif lane_short == 'support' and not redsupport: redsupport = champ
    # one row per match
    row = {
        'matchId': mid,
        'gameDuration': info.get('gameDuration', 0),  # length in seconds
        'bluetop': bluetop,
        'bluejg': bluejg,
        'bluemid': bluemid,
        'bluebot': bluebot,
        'bluesupport': bluesupport,
        'redtop': redtop,
        'redjg': redjg,
        'redmid': redmid,
        'redbot': redbot,
        'redsupport': redsupport
    }
    pregame_rows.append(row)

df_pregame = pd.DataFrame(pregame_rows)
pregame_path_parquet = os.path.join(processed_dir, 'lol_pregame_data.parquet')
pregame_path_csv = os.path.join(processed_dir, 'lol_pregame_data_preview.csv')
# Save full + small preview (first 50 rows)
df_pregame.to_parquet(pregame_path_parquet, index=False)
df_pregame.head(50).to_csv(pregame_path_csv, index=False)
print(f"Simplified pregame table saved: {pregame_path_parquet} (rows={len(df_pregame)})")
print('Columns:', list(df_pregame.columns))

Pregame (all lanes): 100%|██████████| 45/45 [00:00<00:00, 1063.85it/s]

Simplified pregame table saved: data/processed\lol_pregame_data.parquet (rows=45)
Columns: ['matchId', 'gameDuration', 'bluetop', 'bluejg', 'bluemid', 'bluebot', 'bluesupport', 'redtop', 'redjg', 'redmid', 'redbot', 'redsupport']


## Ten Minute Table

In [59]:
# Process all matches (10-minute table)
raw_dir = 'data/raw'
all_match_files = [f.replace('.json','') for f in os.listdir(raw_dir) if f.endswith('.json')]

print("Number of match JSON files in data/raw:", len([f for f in os.listdir(raw_dir) if f.endswith('.json')]))

MIN_GAME_DURATION = 600  # 10 minutes minimum to include
rows = []
skipped_short = []

for match_id in tqdm(all_match_ids, desc="Processing matches (10m)"):
    match_json = load_match_json(match_id)
    game_duration = match_json['info'].get('gameDuration', 0)
    if game_duration < MIN_GAME_DURATION:  # ignore super short games
        skipped_short.append((match_id, game_duration))
        continue
    
    timeline_json = load_timeline_json(match_id)  # need events for early stats
    stats_10min = aggregate_participant_stats_at_10min(match_json, timeline_json)  # custom helper
    teams = match_json['info']['teams']

    # who got first objective
    def first_team(obj_key):
        try:
            if teams[0]['objectives'][obj_key]['first']:
                return 'blue'
            if teams[1]['objectives'][obj_key]['first']:
                return 'red'
        except Exception:
            return None
        return None

    # base row
    row = {
        'matchId': match_id,
        'queueId': match_json['info'].get('queueId'),
        'gameDuration': game_duration,
        'blue_win': int(match_json['info']['teams'][0]['win']),  # 1 if blue won
        'first_tower': first_team('tower'),
        'first_dragon': first_team('dragon'),
        'first_herald': first_team('riftHerald'),
        'first_grub': first_team('horde')  # void grub (horde)
    }
    # participantId lists (1..10) split by side
    blue_ids = [p['participantId'] for p in match_json['info']['participants'][:5]]
    red_ids = [p['participantId'] for p in match_json['info']['participants'][5:]]
    # aggregate team sums at 10m
    row.update({
        'blue_gold_10': sum(stats_10min[pid]['totalGold'] for pid in blue_ids),
        'red_gold_10': sum(stats_10min[pid]['totalGold'] for pid in red_ids),
        'blue_xp_10': sum(stats_10min[pid]['xp'] for pid in blue_ids),
        'red_xp_10': sum(stats_10min[pid]['xp'] for pid in red_ids),
        'blue_cs_10': sum(stats_10min[pid]['minionsKilled'] for pid in blue_ids),
        'red_cs_10': sum(stats_10min[pid]['minionsKilled'] for pid in red_ids),
        'blue_kills_10': sum(stats_10min[pid]['kills'] for pid in blue_ids),
        'red_kills_10': sum(stats_10min[pid]['kills'] for pid in red_ids),
    })
    # diffs (blue - red)
    row['gold_diff_10'] = row['blue_gold_10'] - row['red_gold_10']
    row['cs_diff_10'] = row['blue_cs_10'] - row['red_cs_10']
    row['xp_diff_10'] = row['blue_xp_10'] - row['red_xp_10']
    row['kills_diff_10'] = row['blue_kills_10'] - row['red_kills_10']
    rows.append(row)

df_10m = pd.DataFrame(rows)
print(f"Rows in 10m DataFrame (>= {MIN_GAME_DURATION}s):", df_10m.shape[0])
if skipped_short:
    print(f"Skipped {len(skipped_short)} short games (<{MIN_GAME_DURATION}s):")
    for mid, dur in skipped_short[:10]:
        print(f"  - {mid} (duration={dur}s)")
    if len(skipped_short) > 10:
        print(f"  ... {len(skipped_short)-10} more")
print(df_10m[['matchId','gameDuration','first_tower','first_dragon','first_herald','first_grub']].head())

# Save outputs (full + small preview)
tenmin_parquet = os.path.join(processed_dir, 'lol_10min_data.parquet')
tenmin_csv = os.path.join(processed_dir, 'lol_10min_data_preview.csv')
df_10m.to_parquet(tenmin_parquet, index=False)
df_10m.head(50).to_csv(tenmin_csv, index=False)
print('Saved 10-minute dataset files (Parquet + preview CSV).')

Number of match JSON files in data/raw: 90


Processing matches (10m): 100%|██████████| 20/20 [00:00<00:00, 113.66it/s]

Rows in 10m DataFrame (>= 600s): 19
Skipped 1 short games (<600s):
  - EUN1_3832486067 (duration=117s)
           matchId  gameDuration first_tower first_dragon first_herald  \
0  EUN1_3834387321          1995        blue         blue         blue   
1  EUN1_3834185747          2567         red         blue          red   
2  EUN1_3834161960          1674        blue         blue         blue   
3  EUN1_3833729445          2134        blue          red         blue   
4  EUN1_3833574230          1906         red          red         blue   

  first_grub  
0       blue  
1        red  
2       blue  
3        red  
4        red  
Saved 10-minute dataset files (Parquet + preview CSV).
